### instalação de pacotes [executar]

In [ ]:
!pip install pdfminer.six
!pip3 install unidecode
!python -m spacy download pt_core_news_sm

### Imports [executar]

In [ ]:
import os
import unidecode
import re
from pdfminer.high_level import extract_text
from glob import glob
import spacy
import pt_core_news_sm

# Explorando os PDFs
- adaptando código para leitura de todos pdfs

In [ ]:
cct_com_problemas = '/content/drive/MyDrive/Hacka datathONS/CCTs com problemas'

### métodos para extrair e limpar texto dos PDFs

In [ ]:
def extract_from_pdf(file_path) -> str:
  try:
    return extract_text(file_path)
  except ValueError:
    return file_path
    
def to_ascii(text):
  '''
  padroniza o texto [...]
  '''
  outstring = ''
  for i in range(len(text)):
    outstring = outstring + unidecode.unidecode(text[i])
  return outstring

def clean_text(text):
  '''
  transforma em lowercase e remove quebras de linha e outros caracteres
  '''
  text = str.lower(text)
  pattern=r'[^a-zA-z0-9\s]'
  text=re.sub(pattern,'',text)
  text=re.sub('_', '', text)
  text=re.sub('\n', '', text)
  text=re.sub(' +', ' ', text)
  return text

### teste

In [ ]:
filenames = os.listdir(cct_com_problemas)

text = extract_from_pdf(os.path.join(cct_com_problemas, filenames[0]))
text = to_ascii(text)
text = clean_text(text)

___
# Validações - Grupo 1

- CNPJs
- Razão social
- endereços
___

- ONS
  - Buscar se o CNPJ da ONS aparece no texto.
    - obs.: se aparecer pelo menos uma vez corretamente então estará ok.
  e se tiver um certo e outro errado?

- Transmissora
- Usuárias
- Usuárias anuentes


Possibilidades:
(criar tokens e calcular similaridade entre endereços, em vez de match)

- extrair todos CNPJs e buscar na base de quem são
  - o CNPJ existe na base?
  - a razão social deste CNPJ na base bate com o do doc?
  - o endereço deste CNPJ na base bate com o do doc?



In [ ]:
def extract_all_cnpj(doc: str) -> list:
  pass

def get_razao_social(cnpj: str) -> str:
  pass

def get_endereco(cnpj: str) -> str:
  pass

cnpjs = extract_all_cnpj(doc)
for it in cnpjs:
  razao = get_razao_social(it)
  endereco = get_endereco(it)
  # existe algo parecido/igual no doc?

In [ ]:
CNPJ_ONS = '02.831.210/0001-57'
CNPJ_PATTERN = '[0-9]{2}\.?[0-9]{3}\.?[0-9]{3}\/?[0-9]{4}\-?[0-9]{2}'

def valida_cnpj_ons(texto):
    cnpjs = re.findall(pattern=CNPJ_PATTERN, string=texto)
    cnpjs = [cnpj.replace(' ', '') for cnpj in cnpjs]
    
    if CNPJ_ONS in cnpjs:
        return True
    else:
        False

Teste da função de validação do CNPJ do ONS

In [ ]:
dir_contratos_sem_problemas = './Contratos_sem_problemas/CCTs assinados digitalmente' # Diretório onde estão os contratos
contratos_sem_problemas = glob(f'{dir_contratos_sem_problemas}/*.pdf')                # Lista os pdf no diretório de dados

for i, arquivo in enumerate(contratos_sem_problemas[:10]):
    texto  = extract_text(arquivo).replace('\n', '').replace(' ', '')
    
    if valida_cnpj_ons(texto):
        print('CNPJ do ONS encontrado no arquivo:', arquivo)
    else:
        print('CNPJ DO ONS NÃO ENCONTRADO NO ARQUIVO:', arquivo)


CNPJ do ONS encontrado no arquivo: ./Contratos_sem_problemas/CCTs assinados digitalmente\CCT-2000-001-20 CEEE GT X RGE SUL-Assinado.pdf
CNPJ do ONS encontrado no arquivo: ./Contratos_sem_problemas/CCTs assinados digitalmente\CCT-2000-001-21 - CEEEE GT e RGE SUL-Assinado.pdf
CNPJ do ONS encontrado no arquivo: ./Contratos_sem_problemas/CCTs assinados digitalmente\CCT-2000-001-22 - CEEE GT e RGE SUL-Assinado.pdf
CNPJ do ONS encontrado no arquivo: ./Contratos_sem_problemas/CCTs assinados digitalmente\CCT-2000-001-23 - CEEE e RGE SUL (9908114-2000 TA23)-Assinado.pdf
CNPJ do ONS encontrado no arquivo: ./Contratos_sem_problemas/CCTs assinados digitalmente\CCT-2000-001-24 -CEEE T e RGE SUL-Assinado.pdf
CNPJ do ONS encontrado no arquivo: ./Contratos_sem_problemas/CCTs assinados digitalmente\CCT-2000-001-25 - CEEE T e RGE SUL-Assinado.pdf
CNPJ do ONS encontrado no arquivo: ./Contratos_sem_problemas/CCTs assinados digitalmente\CCT-2000-001-26 - CEEE T x RGE SUL-Assinado.pdf
CNPJ do ONS encontrado

Obs: A função foi testada com os 200 primeiros arquivos de contratos sem problemas.

In [ ]:
def find_pattern(text, pattern):
  i = 0
  n = 1
  interesse = []
  while n > 0:
    n = text.find(pattern, i)
    if n > 0:
      interesse.append(n)
    i = n + 1
  return interesse

find_pattern(text, 'razao')


[56568]

## Razão Social


In [ ]:
text

'cct no 0012020 contrato de conexao ao sistema de transmissao cct 0012020 contratantes atlantico concessionaria de transmissao de energia do brasil sa araguaia niquel metais ltda interveniencia operador nacional do sistema eletrico ons instalacoes envolvidas subestacao xinguara ii barramento de 230 kv pagina 1 de 39 \x0ccct no 0012020 sumario t i t u l o i das definicoes aplicaveis ao presente contrato 6 t i t u l o ii do objeto da caracterizacao do objeto do prazo e da vigencia 9 capitulo i do objeto 9 capitulo ii da caracterizacao do objeto 9 capitulo iii do prazo e da vigencia 10 t i t u l o iii da implantacao 10 capitulo i da implantacao das instalacoes de conexao 10 t i t u l o iv taxa de conservacao das instalacoes 18 capitulo i da taxa de conservacao 18 t i t u l o v da fase operacional das instalacoes de conexao 19 capitulo i dos procedimentos de rede e do acordo operativo 19 capitulo ii da operacao e manutencao das instalacoes de conexao 20 capitulo iii adequacoes ou modificac

In [ ]:
nlp = pt_core_news_sm.load()
doc = nlp('CCT No 0012020 CONTRATO DE CONEXAO AO SISTEMA DE TRANSMISSAO CCT 0012020 Contratantes ATLANTICO CONCESSIONARIA DE TRANSMISSAO DE ENERGIA DO BRASIL SA ARAGUAIA NIQUEL METAIS LTDA Interveniencia OPERADOR NACIONAL DO SISTEMA ELETRICO ONS Instalacoes envolvidas SUBESTACAO XINGUARA II Barramento de 230 kV Pagina 1 de 39')
print([(w.text, w.pos_) for w in doc])

[('CCT', 'PROPN'), ('No', 'ADP'), ('0012020', 'NUM'), ('CONTRATO', 'NOUN'), ('DE', 'ADP'), ('CONEXAO', 'PROPN'), ('AO', 'ADP'), ('SISTEMA', 'NOUN'), ('DE', 'ADP'), ('TRANSMISSAO', 'NOUN'), ('CCT', 'PROPN'), ('0012020', 'NUM'), ('Contratantes', 'PROPN'), ('ATLANTICO', 'PROPN'), ('CONCESSIONARIA', 'ADJ'), ('DE', 'ADP'), ('TRANSMISSAO', 'PROPN'), ('DE', 'PROPN'), ('ENERGIA', 'PROPN'), ('DO', 'ADP'), ('BRASIL', 'PROPN'), ('SA', 'PROPN'), ('ARAGUAIA', 'PROPN'), ('NIQUEL', 'PROPN'), ('METAIS', 'VERB'), ('LTDA', 'PROPN'), ('Interveniencia', 'PROPN'), ('OPERADOR', 'PROPN'), ('NACIONAL', 'PROPN'), ('DO', 'ADP'), ('SISTEMA', 'PROPN'), ('ELETRICO', 'NOUN'), ('ONS', 'VERB'), ('Instalacoes', 'PROPN'), ('envolvidas', 'VERB'), ('SUBESTACAO', 'NOUN'), ('XINGUARA', 'ADJ'), ('II', 'PROPN'), ('Barramento', 'PROPN'), ('de', 'ADP'), ('230', 'NUM'), ('kV', 'NUM'), ('Pagina', 'NOUN'), ('1', 'NUM'), ('de', 'ADP'), ('39', 'NUM')]


In [ ]:
[it for it in doc if it.pos_ == 'PROPN']

[CCT,
 CONEXAO,
 CCT,
 Contratantes,
 ATLANTICO,
 TRANSMISSAO,
 DE,
 ENERGIA,
 BRASIL,
 SA,
 ARAGUAIA,
 NIQUEL,
 LTDA,
 Interveniencia,
 OPERADOR,
 NACIONAL,
 SISTEMA,
 Instalacoes,
 II,
 Barramento]